# Library

In [2]:
import pymysql

## DB 접속

In [31]:
# 전역변수 선언부
conn = None
cur = None

# 메인 코드
password = input('비밀번호를 입력하세요')
conn = pymysql.connect(host = 'localhost', port=3306, user='root', password=password, db='futsaldata', charset='utf8')
cur = conn.cursor() # 커서생성

In [5]:
# sql = 'CREATE TABLE IF NOT EXISTS userTable (id char(4), userName char(10), email char(15), birthYear int)' # 테스트 테이블 생성
# cur.execute(sql) # sql문 실행

# conn.commit() # 저장

# conn.close() # 종료

## test_records 테이블 생성

In [10]:
# sql = "CREATE TABLE test_records (ID int(11) DEFAULT NULL, DATE datetime DEFAULT NULL, OPTEAMNAME varchar(100) DEFAULT NULL, QUARTER tinyint(4) DEFAULT NULL, GOAL varchar(3) DEFAULT NULL, ASSIST varchar(3) DEFAULT NULL)" # test_records 테이블 생성
# cur.execute(sql) # sql문 실행

# conn.commit() # 저장

# conn.close() # 종료

## test_records 테이블에 데이터 입력

### 2024-08-23 경기
    8/23 금 용산 싼다 골도움
    참: 핵 삐 트 범 뀨 후 잡 뭉 _8명

    1Q) 3:2 승
    후 범
    핵 삐
    후 삐

    2Q) 2:2 무
    핵 뀨
    후 잡

    3Q) 1:0 승
    뭉 후

    4Q) 5:2 승
    삐 뭉
    삐 잡
    삐 트
    뭉 잡
    잡 삐

    5Q) 2:2 무
    핵 삐
    삐 핵

    6Q) 1:0 승
    뭉 후

    7Q) 1:3 패
    뭉 삐

    8Q) 5:0 승
    잡 핵
    잡 트
    잡 핵
    잡
    잡 뀨

    5승 2무 1패

#### 1쿼터
    1Q) 3:2 승
    후 범
    핵 삐
    후 삐

In [33]:
goal_list = ['후', '핵', '후', '상대', '상대']
assist_list = ['범', '삐', '삐', 'NULL', 'NULL']
datetime_list = ['2024-08-23 22:00:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['1'] * len(goal_list)
id_num = 1
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

6

In [30]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(1,'2024-08-23 22:00:00','fc intentar',1,'후','범')
INSERT INTO test_records VALUES(2,'2024-08-23 22:00:00','fc intentar',1,'핵','삐')
INSERT INTO test_records VALUES(3,'2024-08-23 22:00:00','fc intentar',1,'후','삐')
INSERT INTO test_records VALUES(4,'2024-08-23 22:00:00','fc intentar',1,'상대',NULL)
INSERT INTO test_records VALUES(5,'2024-08-23 22:00:00','fc intentar',1,'상대',NULL)


#### 2쿼터
    2Q) 2:2 무
    핵 뀨
    후 잡

In [34]:
goal_list = ['핵', '후', '상대', '상대']
assist_list = ['뀨', '잡', 'NULL', 'NULL']
datetime_list = ['2024-08-23 22:15:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['2'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

10

In [35]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(6,'2024-08-23 22:15:00','fc intentar',2,'핵','뀨')
INSERT INTO test_records VALUES(7,'2024-08-23 22:15:00','fc intentar',2,'후','잡')
INSERT INTO test_records VALUES(8,'2024-08-23 22:15:00','fc intentar',2,'상대',NULL)
INSERT INTO test_records VALUES(9,'2024-08-23 22:15:00','fc intentar',2,'상대',NULL)


#### 3쿼터
    3Q) 1:0 승
    뭉 후

In [36]:
goal_list = ['뭉']
assist_list = ['후']
datetime_list = ['2024-08-23 22:30:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['3'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

11

In [37]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(10,'2024-08-23 22:30:00','fc intentar',3,'뭉','후')


#### 4쿼터
    4Q) 5:2 승
    삐 뭉
    삐 잡
    삐 트
    뭉 잡
    잡 삐

In [38]:
goal_list = ['삐', '삐', '삐', '뭉', '잡', '상대', '상대']
assist_list = ['뭉', '잡', '트', '잡', '삐', 'NULL', 'NULL']
datetime_list = ['2024-08-23 22:45:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['4'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

18

In [39]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(11,'2024-08-23 22:45:00','fc intentar',4,'삐','뭉')
INSERT INTO test_records VALUES(12,'2024-08-23 22:45:00','fc intentar',4,'삐','잡')
INSERT INTO test_records VALUES(13,'2024-08-23 22:45:00','fc intentar',4,'삐','트')
INSERT INTO test_records VALUES(14,'2024-08-23 22:45:00','fc intentar',4,'뭉','잡')
INSERT INTO test_records VALUES(15,'2024-08-23 22:45:00','fc intentar',4,'잡','삐')
INSERT INTO test_records VALUES(16,'2024-08-23 22:45:00','fc intentar',4,'상대',NULL)
INSERT INTO test_records VALUES(17,'2024-08-23 22:45:00','fc intentar',4,'상대',NULL)


#### 5쿼터
    5Q) 2:2 무
    핵 삐
    삐 핵

In [40]:
goal_list = ['핵', '삐', '상대', '상대']
assist_list = ['삐', '핵', 'NULL', 'NULL']
datetime_list = ['2024-08-23 23:00:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['5'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

22

In [41]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(18,'2024-08-23 23:00:00','fc intentar',5,'핵','삐')
INSERT INTO test_records VALUES(19,'2024-08-23 23:00:00','fc intentar',5,'삐','핵')
INSERT INTO test_records VALUES(20,'2024-08-23 23:00:00','fc intentar',5,'상대',NULL)
INSERT INTO test_records VALUES(21,'2024-08-23 23:00:00','fc intentar',5,'상대',NULL)


#### 6쿼터
    6Q) 1:0 승
    뭉 후

In [42]:
goal_list = ['뭉']
assist_list = ['후']
datetime_list = ['2024-08-23 23:15:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['6'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

23

In [43]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(22,'2024-08-23 23:15:00','fc intentar',6,'뭉','후')


#### 7쿼터
    7Q) 1:3 패
    뭉 삐

In [44]:
goal_list = ['뭉', '상대', '상대', '상대']
assist_list = ['삐', 'NULL', 'NULL', 'NULL']
datetime_list = ['2024-08-23 23:30:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['7'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

27

In [45]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(23,'2024-08-23 23:30:00','fc intentar',7,'뭉','삐')
INSERT INTO test_records VALUES(24,'2024-08-23 23:30:00','fc intentar',7,'상대',NULL)
INSERT INTO test_records VALUES(25,'2024-08-23 23:30:00','fc intentar',7,'상대',NULL)
INSERT INTO test_records VALUES(26,'2024-08-23 23:30:00','fc intentar',7,'상대',NULL)


#### 8쿼터
    8Q) 5:0 승
    잡 핵
    잡 트
    잡 핵
    잡
    잡 뀨

In [46]:
goal_list = ['후', '핵', '후', '상대', '상대']
assist_list = ['범', '삐', '삐', 'NULL', 'NULL']
datetime_list = ['2024-08-23 23:45:00'] * len(goal_list)
opteamname_list = ['fc intentar'] * len(goal_list)
quarter_list = ['8'] * len(goal_list)
id_list = [str(x) for x in range(id_num, id_num + len(goal_list))]
id_num += len(goal_list)
id_num

32

In [47]:
sql = ''
for id, datetime, opteamname, quarter, goal, assist in zip(id_list, datetime_list, opteamname_list, quarter_list, goal_list, assist_list):
    if assist != 'NULL':
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "','" + assist + "')"
        print(sql)
    else:
        sql = "INSERT INTO test_records VALUES(" + id + ",'" + datetime + "','" + opteamname + "'," + quarter + ",'" + goal + "'," + assist + ")"
        print(sql)
    cur.execute(sql) # sql문 실행

INSERT INTO test_records VALUES(27,'2024-08-23 23:45:00','fc intentar',8,'후','범')
INSERT INTO test_records VALUES(28,'2024-08-23 23:45:00','fc intentar',8,'핵','삐')
INSERT INTO test_records VALUES(29,'2024-08-23 23:45:00','fc intentar',8,'후','삐')
INSERT INTO test_records VALUES(30,'2024-08-23 23:45:00','fc intentar',8,'상대',NULL)
INSERT INTO test_records VALUES(31,'2024-08-23 23:45:00','fc intentar',8,'상대',NULL)


In [48]:
conn.commit() # 저장

conn.close() # 종료

# End